-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Understanding Delta Lake Transactions

To provide atomic, durable transactions in the Lakehouse, Delta Lake utilizes transaction logs stored alongside data files. This notebook dives into the transaction logs to demonstrate what information is recorded and explore how the current version of the table is materialized.

## Learning Objectives
By the end of this lessons, student will be able to:
- Describe how Delta Lake tracks and manages transactions
- Define the atomicity and durability guarantees of Delta Lake
- Map transactional metadata back to table operations

## Setup
Run the following script to setup necessary variables and clear out past runs of this notebook.

In [0]:
%run ../Includes/Classroom-Setup-1.3

Creating the database "dbacademy_odl_user_771624_databrickslabs_com_adewd_1_3"

Predefined Paths:
 DA.paths.working_dir: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3
 DA.paths.user_db: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db
 DA.paths.checkpoints: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/_checkpoints

Predefined tables in dbacademy_odl_user_771624_databrickslabs_com_adewd_1_3:
 new_records
 updates

Validating datasets...(1 seconds)

Setup completed in 12 seconds

## Atomic, Durable Transactions in the Lakehouse

Briefly, Delta Lake transactions can be described as the following:
- All write operations commit data changes as Parquet files
- Transactions commit when JSON log files are written
- Logs are stored in a nested directory
- Both data and transaction logs inherit the durability guarantees of the file system

The cloud-based object storage used by Databricks provides the following advantages and guarantees:
- Infinitely scalable
- Affordable
- Availability:  > 99.9%
- Durability: > 99.999999999%

Note that the Databricks platform has SLAs separate from cloud vendor infrastructure, and the guarantees of each cloud vendor differ slightly.

## Create a Table and Insert Records

The cell below contains 4 Delta Lake operations.

In [0]:
%sql

CREATE TABLE bronze 
(id INT, name STRING, value DOUBLE); 

INSERT INTO bronze VALUES (1, "Yve", 1.0);
INSERT INTO bronze VALUES (2, "Omar", 2.5);
INSERT INTO bronze VALUES (3, "Elia", 3.3);

num_affected_rows,num_inserted_rows
1,1


As expected, the table has three rows.

In [0]:
%sql
SELECT * FROM bronze

id,name,value
2,Omar,2.5
3,Elia,3.3
1,Yve,1.0


Reviewing the history allows us to see how operations and versions are linked.

In [0]:
%sql
DESCRIBE HISTORY bronze

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2022-10-26T07:01:58.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(4382774100892958),1024-143331-6vol2yy0,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1063)",null,Databricks-Runtime/10.4.x-scala2.12
2,2022-10-26T07:01:57.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(4382774100892958),1024-143331-6vol2yy0,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1063)",null,Databricks-Runtime/10.4.x-scala2.12
1,2022-10-26T07:01:55.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(4382774100892958),1024-143331-6vol2yy0,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1055)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-10-26T07:01:53.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(4382774100892958),1024-143331-6vol2yy0,null,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12


The transaction log directory is nested under the table directory and contains a log file for each transaction (alongside checksum files and other files used to manage consistency with cloud object storage).

In [0]:
files = dbutils.fs.ls(f"{DA.paths.user_db}/bronze/_delta_log")
display(files)

path,name,size,modificationTime
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000000.crc,00000000000000000000.crc,1945,1666767714000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000000.json,00000000000000000000.json,1007,1666767713000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000001.crc,00000000000000000001.crc,1951,1666767716000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000001.json,00000000000000000001.json,970,1666767715000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000002.crc,00000000000000000002.crc,1951,1666767717000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000002.json,00000000000000000002.json,972,1666767717000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000003.crc,00000000000000000003.crc,1951,1666767718000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000003.json,00000000000000000003.json,972,1666767718000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/__tmp_path_dir/,__tmp_path_dir/,0,1666767718000


The table was created without any data present, so the initial version only contains metadata about the table schema and the user, environment, and time of the transaction.

Note that the transaction log is a simple JSON file, which can be reviewed directly from the file or read with Spark.

In [0]:
display(spark.read.json(f"{DA.paths.user_db}/bronze/_delta_log/00000000000000000000.json"))

commitInfo,metaData,protocol
"List(1024-143331-6vol2yy0, Databricks-Runtime/10.4.x-scala2.12, true, WriteSerializable, List(4382774100892958), CREATE TABLE, List(null, true, [], {}), 1666767713363, 7ee3097e-1a45-4cab-9a34-599f74ee6cbc, 8453215174696142, odl_user_771624@databrickslabs.com)",null,null
null,null,"List(1, 2)"
null,"List(1666767713261, List(parquet), 65e689f4-a1cf-4f64-904e-f148f1937ddb, List(), {""type"":""struct"",""fields"":[{""name"":""id"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""name"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""value"",""type"":""double"",""nullable"":true,""metadata"":{}}]})",null


In [0]:
display(spark.read.json(f"{DA.paths.user_db}/bronze/_delta_log/00000000000000000003.json"))

add,commitInfo
null,"List(1024-143331-6vol2yy0, Databricks-Runtime/10.4.x-scala2.12, true, WriteSerializable, List(4382774100892958), WRITE, List(1, 1063, 1), List(Append, []), 2, 1666767717981, bae03bd0-7a05-4ae6-9cca-e9cd919d8111, 8453215174696142, odl_user_771624@databrickslabs.com)"
"List(true, 1666767717000, part-00000-a49c7590-115b-4b36-a763-3a73ccb3e792-c000.snappy.parquet, 1063, {""numRecords"":1,""minValues"":{""id"":3,""name"":""Elia"",""value"":3.3},""maxValues"":{""id"":3,""name"":""Elia"",""value"":3.3},""nullCount"":{""id"":0,""name"":0,""value"":0}}, List(1666767717000000, 268435456))",null


The cell below leverages a Databricks setting to automatically return the Delta transaction log for the most recent commit version.

In [0]:
def display_delta_log(table, version=None):
    if not version:
        version = spark.conf.get("spark.databricks.delta.lastCommitVersionInSession")
    version_str = str(int(version)).zfill(20)
    file = f"{DA.paths.user_db}/{table}/_delta_log/{version_str}.json"
    print("Showing: "+file)
    display(spark.read.json(file))

Here we'll see that a simple insert transaction results in new files being tracked in the **`add`** column alongside **`commitInfo`**.

In [0]:
display_delta_log("bronze")

Showing: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000003.json

add,commitInfo
null,"List(1024-143331-6vol2yy0, Databricks-Runtime/10.4.x-scala2.12, true, WriteSerializable, List(4382774100892958), WRITE, List(1, 1063, 1), List(Append, []), 2, 1666767717981, bae03bd0-7a05-4ae6-9cca-e9cd919d8111, 8453215174696142, odl_user_771624@databrickslabs.com)"
"List(true, 1666767717000, part-00000-a49c7590-115b-4b36-a763-3a73ccb3e792-c000.snappy.parquet, 1063, {""numRecords"":1,""minValues"":{""id"":3,""name"":""Elia"",""value"":3.3},""maxValues"":{""id"":3,""name"":""Elia"",""value"":3.3},""nullCount"":{""id"":0,""name"":0,""value"":0}}, List(1666767717000000, 268435456))",null


## Appending Data
Note that when appending data, multiple data files may be written as part of a single transaction.

In [0]:
%sql
INSERT INTO bronze
VALUES (4, "Ted", 4.7),
       (5, "Tiffany", 5.5),
       (6, "Vini", 6.3)

num_affected_rows,num_inserted_rows
3,3


The **`add`** column below contains paths and stats for each of the files added to the table.

In [0]:
display_delta_log("bronze")

Showing: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000004.json

add,commitInfo
null,"List(1024-143331-6vol2yy0, Databricks-Runtime/10.4.x-scala2.12, true, WriteSerializable, List(4382774100892958), WRITE, List(3, 3202, 3), List(Append, []), 3, 1666768314637, c3b2df53-8cc5-4d84-8f6b-44e417f2fb5f, 8453215174696142, odl_user_771624@databrickslabs.com)"
"List(true, 1666768314000, part-00000-dc1dc05b-4026-443f-9c97-e5ce6beb0288-c000.snappy.parquet, 1056, {""numRecords"":1,""minValues"":{""id"":4,""name"":""Ted"",""value"":4.7},""maxValues"":{""id"":4,""name"":""Ted"",""value"":4.7},""nullCount"":{""id"":0,""name"":0,""value"":0}}, List(1666768314000000, 268435456))",null
"List(true, 1666768314000, part-00001-49dc6475-7679-41d5-89a3-8ffc69bf7297-c000.snappy.parquet, 1083, {""numRecords"":1,""minValues"":{""id"":5,""name"":""Tiffany"",""value"":5.5},""maxValues"":{""id"":5,""name"":""Tiffany"",""value"":5.5},""nullCount"":{""id"":0,""name"":0,""value"":0}}, List(1666768314000001, 268435456))",null
"List(true, 1666768314000, part-00002-793f1942-53f1-4133-b07e-f4151f38a214-c000.snappy.parquet, 1063, {""numRecords"":1,""minValues"":{""id"":6,""name"":""Vini"",""value"":6.3},""maxValues"":{""id"":6,""name"":""Vini"",""value"":6.3},""nullCount"":{""id"":0,""name"":0,""value"":0}}, List(1666768314000002, 268435456))",null


In [0]:
%sql
SELECT * FROM new_records

id,name,value
7,Viktor,7.4
8,Hiro,8.2
9,Shana,9.9


Most append operations will insert many records into a single file.

In [0]:
%sql
INSERT INTO bronze SELECT * FROM new_records

num_affected_rows,num_inserted_rows
3,3


As expected, multiple records appear in a single data file under the **`add`** column.

In [0]:
display_delta_log("bronze")

Showing: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000005.json

add,commitInfo
null,"List(1024-143331-6vol2yy0, Databricks-Runtime/10.4.x-scala2.12, true, WriteSerializable, List(4382774100892958), WRITE, List(1, 1097, 3), List(Append, []), 4, 1666768410573, dfc1d66d-69ce-46a3-92b0-8e7e813d9cb5, 8453215174696142, odl_user_771624@databrickslabs.com)"
"List(true, 1666768410000, part-00000-17cd3eab-134a-4664-b321-f49f30d34c26-c000.snappy.parquet, 1097, {""numRecords"":3,""minValues"":{""id"":7,""name"":""Hiro"",""value"":7.4},""maxValues"":{""id"":9,""name"":""Viktor"",""value"":9.9},""nullCount"":{""id"":0,""name"":0,""value"":0}}, List(1666768410000000, 268435456))",null


## Deleting Data
Deleting a single record from a file with multiple records will actually result in a file being added.

In [0]:
%sql
select * from bronze

id,name,value
7,Viktor,7.4
8,Hiro,8.2
9,Shana,9.9
5,Tiffany,5.5
2,Omar,2.5
3,Elia,3.3
6,Vini,6.3
4,Ted,4.7
1,Yve,1.0


In [0]:
%sql 
DELETE FROM bronze WHERE name = "Viktor"

num_affected_rows
1


The file added contains the other records that were in the same data file as the record deleted in the transaction. 

In this specific case, the **`remove`** column indicates that the previous file with 3 records is no longer valid; the **`add`** column points to a new file containing only 2 records.

This is the expected behavior, as Delta Lake does not modify data files in place.

In [0]:
display_delta_log("bronze")

Showing: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000006.json

add,commitInfo,remove
null,"List(1024-143331-6vol2yy0, Databricks-Runtime/10.4.x-scala2.12, false, WriteSerializable, List(4382774100892958), DELETE, List(924, 0, 1, 2, 1, 1, 413, 510), List([""(spark_catalog.dbacademy_odl_user_771624_databrickslabs_com_adewd_1_3.bronze.name = 'Viktor')""]), 5, 1666768565744, 4ce1d812-94db-4deb-9ea6-27a77f78ace0, 8453215174696142, odl_user_771624@databrickslabs.com)",null
null,null,"List(true, 1666768565727, true, part-00000-17cd3eab-134a-4664-b321-f49f30d34c26-c000.snappy.parquet, 1097, List(1666768410000000, 268435456))"
"List(true, 1666768565000, part-00000-58aec6bb-7502-466c-a3f5-ad576c97dad9-c000.snappy.parquet, 1074, {""numRecords"":2,""minValues"":{""id"":8,""name"":""Hiro"",""value"":8.2},""maxValues"":{""id"":9,""name"":""Shana"",""value"":9.9},""nullCount"":{""id"":0,""name"":0,""value"":0}}, List(1666768410000000, 268435456))",null,null


## Updating Data
Anytime a record in an existing file is modified, a new file will be added, and the old file will be indicated in the **`remove`** column, as in the update below.

In [0]:
%sql
UPDATE bronze SET name = "Vincent" WHERE id = 6

num_affected_rows
1


Here, both the file removed and the file rewritten contain only the modified record. In production environments, you are unlikely to see a data file containing a single record.

In [0]:
display_delta_log("bronze")

Showing: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000007.json

add,commitInfo,remove
null,"List(1024-143331-6vol2yy0, Databricks-Runtime/10.4.x-scala2.12, false, WriteSerializable, List(4382774100892958), UPDATE, List(981, 0, 1, 0, 1, 1, 838, 138), List((id#10669 = 6)), 6, 1666769652843, a131ec3b-939b-43d2-9633-254828c74611, 8453215174696142, odl_user_771624@databrickslabs.com)",null
null,null,"List(true, 1666769652842, true, part-00002-793f1942-53f1-4133-b07e-f4151f38a214-c000.snappy.parquet, 1063, List(1666768314000002, 268435456))"
"List(true, 1666769652000, part-00000-f70d952d-f7da-423e-b07e-d3c7da30fc8f-c000.snappy.parquet, 1083, {""numRecords"":1,""minValues"":{""id"":6,""name"":""Vincent"",""value"":6.3},""maxValues"":{""id"":6,""name"":""Vincent"",""value"":6.3},""nullCount"":{""id"":0,""name"":0,""value"":0}}, List(1666768314000002, 268435456))",null,null


## Combining Transactions with Merge
The Delta Lake **`MERGE`** syntax allows updates, deletes, and inserts to occur in a single transaction.

In [0]:
%sql
MERGE INTO bronze b
USING updates u
ON b.id=u.id
WHEN MATCHED AND u.type = "update"
  THEN UPDATE SET *
WHEN MATCHED AND u.type = "delete"
  THEN DELETE
WHEN NOT MATCHED AND u.type = "insert"
  THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
3,1,1,1


Looking at the table history, note how **`MERGE`** operations differ from **`DELETE`**, **`UPDATE`**, or **`INSERT`** transactions.

In [0]:
%sql
DESCRIBE HISTORY bronze

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
8,2022-10-26T07:35:14.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,MERGE,"Map(predicate -> (b.id = u.id), matchedPredicates -> [{""predicate"":""(u.type = 'update')"",""actionType"":""update""},{""predicate"":""(u.type = 'delete')"",""actionType"":""delete""}], notMatchedPredicates -> [{""predicate"":""(u.type = 'insert')"",""actionType"":""insert""}])",null,List(4382774100892958),1024-143331-6vol2yy0,7,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 1, numTargetFilesAdded -> 2, executionTimeMs -> 2411, numTargetRowsInserted -> 1, scanTimeMs -> 1602, numTargetRowsUpdated -> 1, numOutputRows -> 2, numTargetChangeFilesAdded -> 0, numSourceRows -> 4, numTargetFilesRemoved -> 2, rewriteTimeMs -> 683)",null,Databricks-Runtime/10.4.x-scala2.12
7,2022-10-26T07:34:12.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,UPDATE,Map(predicate -> (id#10669 = 6)),null,List(4382774100892958),1024-143331-6vol2yy0,6,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 981, scanTimeMs -> 138, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 838)",null,Databricks-Runtime/10.4.x-scala2.12
6,2022-10-26T07:16:05.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,DELETE,"Map(predicate -> [""(spark_catalog.dbacademy_odl_user_771624_databrickslabs_com_adewd_1_3.bronze.name = 'Viktor')""])",null,List(4382774100892958),1024-143331-6vol2yy0,5,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 2, numAddedChangeFiles -> 0, executionTimeMs -> 924, numDeletedRows -> 1, scanTimeMs -> 510, numAddedFiles -> 1, rewriteTimeMs -> 413)",null,Databricks-Runtime/10.4.x-scala2.12
5,2022-10-26T07:13:30.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(4382774100892958),1024-143331-6vol2yy0,4,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 1097)",null,Databricks-Runtime/10.4.x-scala2.12
4,2022-10-26T07:11:54.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(4382774100892958),1024-143331-6vol2yy0,3,WriteSerializable,true,"Map(numFiles -> 3, numOutputRows -> 3, numOutputBytes -> 3202)",null,Databricks-Runtime/10.4.x-scala2.12
3,2022-10-26T07:01:58.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(4382774100892958),1024-143331-6vol2yy0,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1063)",null,Databricks-Runtime/10.4.x-scala2.12
2,2022-10-26T07:01:57.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(4382774100892958),1024-143331-6vol2yy0,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1063)",null,Databricks-Runtime/10.4.x-scala2.12
1,2022-10-26T07:01:55.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(4382774100892958),1024-143331-6vol2yy0,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1055)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-10-26T07:01:53.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(4382774100892958),1024-143331-6vol2yy0,null,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12


The **`operationMetrics`** and **`operationParameters`** within the **`commitInfo`** describe the source data, the parameters for the query, and all the results of the transaction.

Use the query below to find:
- How many rows were in the **`updates`** table
- How many records were inserted
- How many records were deleted
- How many records were updated
- The total number of files removed
- The total number of files added

In [0]:
display_delta_log("bronze")

Showing: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000008.json

add,commitInfo,remove
null,"List(1024-143331-6vol2yy0, Databricks-Runtime/10.4.x-scala2.12, false, WriteSerializable, List(4382774100892958), MERGE, List(2411, 2, 4, 0, 2, 2, 0, 1, 1, 1, 683, 1602), List([{""predicate"":""(u.type = 'update')"",""actionType"":""update""},{""predicate"":""(u.type = 'delete')"",""actionType"":""delete""}], [{""predicate"":""(u.type = 'insert')"",""actionType"":""insert""}], (b.id = u.id)), 7, 1666769714605, 9b4a4ec9-ec32-42c3-ae72-e6727ac8391b, 8453215174696142, odl_user_771624@databrickslabs.com)",null
null,null,"List(true, 1666769714501, true, part-00000-a49c7590-115b-4b36-a763-3a73ccb3e792-c000.snappy.parquet, 1063, List(1666767717000000, 268435456))"
null,null,"List(true, 1666769714501, true, part-00000-e11f1709-c07c-4da2-965c-99c343883c99-c000.snappy.parquet, 1063, List(1666767716000000, 268435456))"
"List(true, 1666769714000, part-00000-97eee630-ca85-4b5c-aa99-018fa1adc89a-c000.snappy.parquet, 1063, {""numRecords"":1,""minValues"":{""id"":2,""name"":""Omar"",""value"":15.2},""maxValues"":{""id"":2,""name"":""Omar"",""value"":15.2},""nullCount"":{""id"":0,""name"":0,""value"":0}}, List(1666769714000000, 268435456))",null,null
"List(true, 1666769714000, part-00001-695bf85b-92e6-42ee-8f00-52740f7c1010-c000.snappy.parquet, 1063, {""numRecords"":1,""minValues"":{""id"":10,""name"":""Blue"",""value"":7.7},""maxValues"":{""id"":10,""name"":""Blue"",""value"":7.7},""nullCount"":{""id"":0,""name"":0,""value"":0}}, List(1666769714000001, 268435456))",null,null


## Table Utility Functions
Delta Lake table utility functions modify the metadata of a table without changing underlying data.

In [0]:
%sql
ALTER TABLE bronze
ALTER COLUMN name
COMMENT "User first name"

Files will not be marked as added or removed, as only the metadata has been updated.

In [0]:
display_delta_log("bronze")

Showing: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000009.json

commitInfo,metaData
"List(1024-143331-6vol2yy0, Databricks-Runtime/10.4.x-scala2.12, true, WriteSerializable, List(4382774100892958), CHANGE COLUMN, List({""name"":""name"",""type"":""string"",""nullable"":true,""metadata"":{""comment"":""User first name""}}), 8, 1666769949159, ec08d080-8729-485e-ae51-fe9c225f1d56, 8453215174696142, odl_user_771624@databrickslabs.com)",null
null,"List(1666767713261, List(parquet), 65e689f4-a1cf-4f64-904e-f148f1937ddb, List(), {""type"":""struct"",""fields"":[{""name"":""id"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""name"",""type"":""string"",""nullable"":true,""metadata"":{""comment"":""User first name""}},{""name"":""value"",""type"":""double"",""nullable"":true,""metadata"":{}}]})"


## File Compaction
Running **`OPTIMIZE`** on a table will compact small files toward the target file size.

In [0]:
%sql
OPTIMIZE bronze

path,metrics
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze,"List(1, 7, List(1213, 1213, 1213.0, 1, 1213), List(1055, 1083, 1068.142857142857, 7, 7477), 0, null, 1, 7, 0, true)"


During file compaction, many files should be marked as removed, while a smaller number of files will be marked as added.

No data or files are deleted during this operation; as always, adding files to the **`remove`** column means they will not be used when querying the current version of the table, but does not permanently delete them from the underlying storage.

In [0]:
display_delta_log("bronze")

Showing: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000010.json

add,commitInfo,remove
null,"List(1024-143331-6vol2yy0, Databricks-Runtime/10.4.x-scala2.12, false, SnapshotIsolation, List(4382774100892958), OPTIMIZE, List(1213, 1213, 1213, 1, 7477, 7, 1213, 1213, 1213), List(false, 0, [], []), 9, 1666769975698, be0bee25-940a-4a35-92d4-0ca4877ba0b2, 8453215174696142, odl_user_771624@databrickslabs.com)",null
null,null,"List(false, 1666769974963, true, part-00000-f2ef58f4-607f-4ae2-bbcc-63a6575ba655-c000.snappy.parquet, 1055, List(1666767715000000, 268435456))"
null,null,"List(false, 1666769974963, true, part-00000-dc1dc05b-4026-443f-9c97-e5ce6beb0288-c000.snappy.parquet, 1056, List(1666768314000000, 268435456))"
null,null,"List(false, 1666769974963, true, part-00001-49dc6475-7679-41d5-89a3-8ffc69bf7297-c000.snappy.parquet, 1083, List(1666768314000001, 268435456))"
null,null,"List(false, 1666769974963, true, part-00000-f70d952d-f7da-423e-b07e-d3c7da30fc8f-c000.snappy.parquet, 1083, List(1666768314000002, 268435456))"
null,null,"List(false, 1666769974963, true, part-00000-58aec6bb-7502-466c-a3f5-ad576c97dad9-c000.snappy.parquet, 1074, List(1666768410000000, 268435456))"
null,null,"List(false, 1666769974963, true, part-00000-97eee630-ca85-4b5c-aa99-018fa1adc89a-c000.snappy.parquet, 1063, List(1666769714000000, 268435456))"
null,null,"List(false, 1666769974963, true, part-00001-695bf85b-92e6-42ee-8f00-52740f7c1010-c000.snappy.parquet, 1063, List(1666769714000001, 268435456))"
"List(false, 1666769975000, part-00000-b415de6e-9a9d-459c-be8f-f70f9c6b64bd-c000.snappy.parquet, 1213, {""numRecords"":8,""minValues"":{""id"":1,""name"":""Blue"",""value"":1.0},""maxValues"":{""id"":10,""name"":""Yve"",""value"":15.2},""nullCount"":{""id"":0,""name"":0,""value"":0}}, List(1666767715000000, 268435456))",null,null


## Transaction Log Checkpoints
Databricks will automatically create Parquet checkpoint files at fixed intervals to accelerate the resolution of the current table state.

Version 10 of the table should have both a **`.json`** and a **`.checkpoint.parquet`** file associated with it.

In [0]:
files = dbutils.fs.ls(f"{DA.paths.user_db}/bronze/_delta_log")
display(files)

path,name,size,modificationTime
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000000.crc,00000000000000000000.crc,1945,1666767714000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000000.json,00000000000000000000.json,1007,1666767713000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000001.crc,00000000000000000001.crc,1951,1666767716000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000001.json,00000000000000000001.json,970,1666767715000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000002.crc,00000000000000000002.crc,1951,1666767717000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000002.json,00000000000000000002.json,972,1666767717000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000003.crc,00000000000000000003.crc,1951,1666767718000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000003.json,00000000000000000003.json,972,1666767718000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000004.crc,00000000000000000004.crc,1951,1666768315000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000004.json,00000000000000000004.json,1860,1666768314000


Rather than only showing the operations of the most recent transaction, this checkpoint file condenses all of the **`add`** and **`remove`** instructions and valid **`metaData`** into a single file.

This means that rather than loading many JSON files and comparing files listed in the **`add`** and **`remove`** columns to find those data files that currently represent the valid table version, a single file can be loaded that fully describes the table state.

Transactions after a checkpoint leverage this starting point, resolved new info from JSON files with the instructions from this Parquet snapshot.

In [0]:
display(spark.read.parquet(f"{DA.paths.user_db}/bronze/_delta_log/00000000000000000010.checkpoint.parquet"))

txn,add,remove,metaData,protocol
null,null,null,null,"List(1, 2)"
null,null,null,"List(65e689f4-a1cf-4f64-904e-f148f1937ddb, null, null, List(parquet, Map()), {""type"":""struct"",""fields"":[{""name"":""id"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""name"",""type"":""string"",""nullable"":true,""metadata"":{""comment"":""User first name""}},{""name"":""value"",""type"":""double"",""nullable"":true,""metadata"":{}}]}, List(), Map(), 1666767713261)",null
null,null,"List(part-00000-17cd3eab-134a-4664-b321-f49f30d34c26-c000.snappy.parquet, 1666768565727, false, true, Map(), 1097, Map(INSERTION_TIME -> 1666768410000000, OPTIMIZE_TARGET_SIZE -> 268435456))",null,null
null,null,"List(part-00000-58aec6bb-7502-466c-a3f5-ad576c97dad9-c000.snappy.parquet, 1666769974963, false, true, Map(), 1074, Map(INSERTION_TIME -> 1666768410000000, OPTIMIZE_TARGET_SIZE -> 268435456))",null,null
null,null,"List(part-00000-97eee630-ca85-4b5c-aa99-018fa1adc89a-c000.snappy.parquet, 1666769974963, false, true, Map(), 1063, Map(INSERTION_TIME -> 1666769714000000, OPTIMIZE_TARGET_SIZE -> 268435456))",null,null
null,null,"List(part-00000-a49c7590-115b-4b36-a763-3a73ccb3e792-c000.snappy.parquet, 1666769714501, false, true, Map(), 1063, Map(INSERTION_TIME -> 1666767717000000, OPTIMIZE_TARGET_SIZE -> 268435456))",null,null
null,"List(part-00000-b415de6e-9a9d-459c-be8f-f70f9c6b64bd-c000.snappy.parquet, Map(), 1213, 1666769975000, false, Map(INSERTION_TIME -> 1666767715000000, OPTIMIZE_TARGET_SIZE -> 268435456), {""numRecords"":8,""minValues"":{""id"":1,""name"":""Blue"",""value"":1.0},""maxValues"":{""id"":10,""name"":""Yve"",""value"":15.2},""nullCount"":{""id"":0,""name"":0,""value"":0}}, List(8, List(1, Blue, 1.0), List(10, Yve, 15.2), List(0, 0, 0)))",null,null,null
null,null,"List(part-00000-dc1dc05b-4026-443f-9c97-e5ce6beb0288-c000.snappy.parquet, 1666769974963, false, true, Map(), 1056, Map(INSERTION_TIME -> 1666768314000000, OPTIMIZE_TARGET_SIZE -> 268435456))",null,null
null,null,"List(part-00000-e11f1709-c07c-4da2-965c-99c343883c99-c000.snappy.parquet, 1666769714501, false, true, Map(), 1063, Map(INSERTION_TIME -> 1666767716000000, OPTIMIZE_TARGET_SIZE -> 268435456))",null,null
null,null,"List(part-00000-f2ef58f4-607f-4ae2-bbcc-63a6575ba655-c000.snappy.parquet, 1666769974963, false, true, Map(), 1055, Map(INSERTION_TIME -> 1666767715000000, OPTIMIZE_TARGET_SIZE -> 268435456))",null,null


Note that all of the data files in both the **`add`** and **`remove`** columns are still present in the table directory.

In [0]:
files = dbutils.fs.ls(f"{DA.paths.user_db}/bronze")
display(files)

path,name,size,modificationTime
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/,_delta_log/,0,1666769977000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/part-00000-17cd3eab-134a-4664-b321-f49f30d34c26-c000.snappy.parquet,part-00000-17cd3eab-134a-4664-b321-f49f30d34c26-c000.snappy.parquet,1097,1666768410000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/part-00000-58aec6bb-7502-466c-a3f5-ad576c97dad9-c000.snappy.parquet,part-00000-58aec6bb-7502-466c-a3f5-ad576c97dad9-c000.snappy.parquet,1074,1666768565000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/part-00000-97eee630-ca85-4b5c-aa99-018fa1adc89a-c000.snappy.parquet,part-00000-97eee630-ca85-4b5c-aa99-018fa1adc89a-c000.snappy.parquet,1063,1666769714000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/part-00000-a49c7590-115b-4b36-a763-3a73ccb3e792-c000.snappy.parquet,part-00000-a49c7590-115b-4b36-a763-3a73ccb3e792-c000.snappy.parquet,1063,1666767717000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/part-00000-b415de6e-9a9d-459c-be8f-f70f9c6b64bd-c000.snappy.parquet,part-00000-b415de6e-9a9d-459c-be8f-f70f9c6b64bd-c000.snappy.parquet,1213,1666769975000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/part-00000-dc1dc05b-4026-443f-9c97-e5ce6beb0288-c000.snappy.parquet,part-00000-dc1dc05b-4026-443f-9c97-e5ce6beb0288-c000.snappy.parquet,1056,1666768314000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/part-00000-e11f1709-c07c-4da2-965c-99c343883c99-c000.snappy.parquet,part-00000-e11f1709-c07c-4da2-965c-99c343883c99-c000.snappy.parquet,1063,1666767716000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/part-00000-f2ef58f4-607f-4ae2-bbcc-63a6575ba655-c000.snappy.parquet,part-00000-f2ef58f4-607f-4ae2-bbcc-63a6575ba655-c000.snappy.parquet,1055,1666767715000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/part-00000-f70d952d-f7da-423e-b07e-d3c7da30fc8f-c000.snappy.parquet,part-00000-f70d952d-f7da-423e-b07e-d3c7da30fc8f-c000.snappy.parquet,1083,1666769652000


## Cleaning Up Stale Data Files
Executing **`VACUUM`** performs garbage cleanup on this directory. By default, a retention threshold of 7 days will be enforced; here it is overridden to demonstrate permanent removal of data. Manually setting **`spark.databricks.delta.vacuum.logging.enabled`** to **`True`** ensures that this operation is also recorded in the transaction log. 

**NOTE**: Vacuuming a production table with a short retention can lead to data corruption and/or failure of long-running queries.

In [0]:
spark.conf.set("spark.databricks.delta.vacuum.logging.enabled", True)
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)
spark.sql("VACUUM bronze RETAIN 0 HOURS")
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", True)

As expected, a single file remains.

In [0]:
files = dbutils.fs.ls(f"{DA.paths.user_db}/bronze")
display(files)

path,name,size,modificationTime
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/,_delta_log/,0,1666770197000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/part-00000-b415de6e-9a9d-459c-be8f-f70f9c6b64bd-c000.snappy.parquet,part-00000-b415de6e-9a9d-459c-be8f-f70f9c6b64bd-c000.snappy.parquet,1213,1666769975000


Note that both the start and end of the **`VACUUM`** operation are recorded in the history.

In [0]:
%sql
DESCRIBE HISTORY bronze

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
12,2022-10-26T07:43:15.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,VACUUM END,Map(status -> COMPLETED),null,List(4382774100892958),1024-143331-6vol2yy0,11,SnapshotIsolation,true,"Map(numDeletedFiles -> 11, numVacuumedDirectories -> 1)",null,Databricks-Runtime/10.4.x-scala2.12
11,2022-10-26T07:43:12.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,VACUUM START,"Map(retentionCheckEnabled -> false, specifiedRetentionMillis -> 0, defaultRetentionMillis -> 604800000)",null,List(4382774100892958),1024-143331-6vol2yy0,10,SnapshotIsolation,true,Map(numFilesToDelete -> 11),null,Databricks-Runtime/10.4.x-scala2.12
10,2022-10-26T07:39:35.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [], batchId -> 0, auto -> false)",null,List(4382774100892958),1024-143331-6vol2yy0,9,SnapshotIsolation,false,"Map(numRemovedFiles -> 7, numRemovedBytes -> 7477, p25FileSize -> 1213, minFileSize -> 1213, numAddedFiles -> 1, maxFileSize -> 1213, p75FileSize -> 1213, p50FileSize -> 1213, numAddedBytes -> 1213)",null,Databricks-Runtime/10.4.x-scala2.12
9,2022-10-26T07:39:09.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,CHANGE COLUMN,"Map(column -> {""name"":""name"",""type"":""string"",""nullable"":true,""metadata"":{""comment"":""User first name""}})",null,List(4382774100892958),1024-143331-6vol2yy0,8,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12
8,2022-10-26T07:35:14.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,MERGE,"Map(predicate -> (b.id = u.id), matchedPredicates -> [{""predicate"":""(u.type = 'update')"",""actionType"":""update""},{""predicate"":""(u.type = 'delete')"",""actionType"":""delete""}], notMatchedPredicates -> [{""predicate"":""(u.type = 'insert')"",""actionType"":""insert""}])",null,List(4382774100892958),1024-143331-6vol2yy0,7,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 1, numTargetFilesAdded -> 2, executionTimeMs -> 2411, numTargetRowsInserted -> 1, scanTimeMs -> 1602, numTargetRowsUpdated -> 1, numOutputRows -> 2, numTargetChangeFilesAdded -> 0, numSourceRows -> 4, numTargetFilesRemoved -> 2, rewriteTimeMs -> 683)",null,Databricks-Runtime/10.4.x-scala2.12
7,2022-10-26T07:34:12.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,UPDATE,Map(predicate -> (id#10669 = 6)),null,List(4382774100892958),1024-143331-6vol2yy0,6,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 981, scanTimeMs -> 138, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 838)",null,Databricks-Runtime/10.4.x-scala2.12
6,2022-10-26T07:16:05.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,DELETE,"Map(predicate -> [""(spark_catalog.dbacademy_odl_user_771624_databrickslabs_com_adewd_1_3.bronze.name = 'Viktor')""])",null,List(4382774100892958),1024-143331-6vol2yy0,5,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 2, numAddedChangeFiles -> 0, executionTimeMs -> 924, numDeletedRows -> 1, scanTimeMs -> 510, numAddedFiles -> 1, rewriteTimeMs -> 413)",null,Databricks-Runtime/10.4.x-scala2.12
5,2022-10-26T07:13:30.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(4382774100892958),1024-143331-6vol2yy0,4,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 1097)",null,Databricks-Runtime/10.4.x-scala2.12
4,2022-10-26T07:11:54.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(4382774100892958),1024-143331-6vol2yy0,3,WriteSerializable,true,"Map(numFiles -> 3, numOutputRows -> 3, numOutputBytes -> 3202)",null,Databricks-Runtime/10.4.x-scala2.12
3,2022-10-26T07:01:58.000+0000,8

The **`VACUUM START`** version will record the number of files to be deleted, but does not contain a list of file names.

Once deleted, previous versions of the table relying on these files are no longer accessible.

In [0]:
display_delta_log("bronze", 11)

Showing: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3/1_3.db/bronze/_delta_log/00000000000000000011.json

commitInfo
"List(1024-143331-6vol2yy0, Databricks-Runtime/10.4.x-scala2.12, true, SnapshotIsolation, List(4382774100892958), VACUUM START, List(11), List(604800000, false, 0), 10, 1666770192775, 377cbba6-c2b4-4bb8-8779-9842ca28c79a, 8453215174696142, odl_user_771624@databrickslabs.com)"


Additional reading is available in this <a href="https://databricks.com/blog/2019/08/21/diving-into-delta-lake-unpacking-the-transaction-log.html" target="_blank">blog post</a>.

Run the following cell to delete the tables and files associated with this lesson.

In [0]:
DA.cleanup()

Dropping the database "dbacademy_odl_user_771624_databrickslabs_com_adewd_1_3"
Removing the working directory "dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.3"
Validating datasets...(1 seconds)

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>